<a href="https://colab.research.google.com/github/delphi12/VirtualAssistant_Project/blob/main/SuperGlue_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
pip install datasets transformers > /dev/null

In [3]:
squad_v2 = False
model_checkpoint = "bert-large-uncased-whole-word-masking-finetuned-squad"
batch_size = 32

In [4]:
from datasets import load_dataset, load_metric

In [5]:
datasets = load_dataset("super_glue", "multirc")

Generating train split:   0%|          | 0/27243 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4848 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9693 [00:00<?, ? examples/s]

Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/multirc/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
datasets["train"]

Dataset({
    features: ['paragraph', 'question', 'answer', 'idx', 'label'],
    num_rows: 27243
})

In [7]:
datasets["validation"] = datasets["train"]
datasets["validation"] = datasets["validation"].select([i for i in range(0,300)])
datasets["train"] = datasets["train"].select([i for i in range(300,2019)])

In [10]:
len(datasets["train"])

1719

In [9]:
datasets["train"][0]

{'paragraph': 'The side of Malaquez\'s parcel gave way to reveal a greenmunk caught in a sheen of solid air. Bits of leaf mold flew from under his feet as he ran to greet a friend or a bringer of food. Tasha oohed in awe. I said, "Frodo\'s been visiting you, eh?" Malaquez said, "Your pet?" "Hardly. He lives around here somewhere. I suppose he was attracted to the commotion up the hill." "Ah," Malaquez said. "Why \'Frodo\'?" Tasha said, "A little fellow with big, furry feet. What else could he be called?" She handed the sculpture to me. I almost dropped it; I expected it to weigh no more than a holo. "Heavy," I said, as if he might not have known. He laughed. "My last piece was of four old Undersiders crouched around a trash fire. Be glad someone didn\'t toss that to you." He spoke of his art with the enthusiasm of a seven-year-old. "Um, I should wait to importune you, but..." ',
 'question': "Is Frodo the narrator's pet?",
 'answer': 'Frodo is a pet',
 'idx': {'paragraph': 5, 'question

In [11]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [12]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [14]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
max_length = 384
doc_stride = 128

In [16]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["paragraph"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [17]:
len(tokenizer(example["question"], example["paragraph"])["input_ids"])

446

In [18]:
len(tokenizer(example["question"], example["paragraph"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [19]:
tokenized_example = tokenizer(
    example["question"],
    example["paragraph"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [20]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 205]

In [21]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] list some of the effects of the 2010 deepwater horizon explosion [SEP] ( cnn ) - - declaring 2010 " the best year in safety performance in our company's history, " transocean ltd., owner of the gulf of mexico oil rig that exploded, killing 11 workers, has awarded its top executives hefty bonuses and raises, according to a recent filing with the u. s. securities and exchange commission. that includes a $ 200, 000 salary increase for transocean president and chief executive officer steven l. newman, whose base salary will increase from $ 900, 000 to $ 1. 1 million, according to the sec report. newman's bonus was $ 374, 062, the report states. newman also has a $ 5. 4 million long - term compensation package the company awarded him upon his appointment as ceo in march 2010, according to the sec filing. the latest cash awards are based in part on the company's " performance under safety, " the transocean filing states. " notwithstanding the tragic loss of life in the gulf of mexico, 

In [22]:
tokenized_example = tokenizer(
    example["question"],
    example["paragraph"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 9), (10, 12), (13, 16), (17, 24), (25, 27), (28, 31), (32, 36), (37, 41), (41, 46), (47, 54), (55, 64), (0, 0), (0, 1), (1, 4), (4, 5), (6, 7), (7, 8), (9, 18), (19, 23), (24, 25), (25, 28), (29, 33), (34, 38), (39, 41), (42, 48), (49, 60), (61, 63), (64, 67), (68, 75), (75, 76), (76, 77), (78, 85), (85, 86), (86, 87), (88, 93), (93, 95), (95, 98), (99, 102), (102, 103), (103, 104), (105, 110), (111, 113), (114, 117), (118, 122), (123, 125), (126, 132), (133, 136), (137, 140), (141, 145), (146, 154), (154, 155), (156, 163), (164, 166), (167, 174), (174, 175), (176, 179), (180, 187), (188, 191), (192, 195), (196, 206), (207, 209), (209, 211), (211, 212), (213, 220), (221, 224), (225, 231), (231, 232), (233, 242), (243, 245), (246, 247), (248, 254), (255, 261), (262, 266), (267, 270), (271, 272), (272, 273), (273, 274), (274, 275), (276, 286), (287, 290), (291, 299), (300, 310), (310, 311), (312, 316), (317, 325), (326, 327), (328, 329), (329, 332), (332, 333), (333,

In [23]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

list List


In [24]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [25]:
answers = example["answer"]
start_char = example["idx"]["answer"]
end_char = start_char + len(answers)

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

111 113


In [26]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers)

will increase from
Injured 17 workers


In [27]:
pad_on_right = tokenizer.padding_side == "right"

In [28]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "paragraph"],
        examples["paragraph" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        #example["idx"]["answer"]
        # If no answers are given, set the cls_index as answer.
        if (example["idx"]["answer"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = example["idx"]["answer"]
            end_char = start_char + len(answers)

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [29]:
features = prepare_train_features(datasets['train'][:5])


In [30]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/1719 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/9693 [00:00<?, ? examples/s]

In [31]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [32]:
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [33]:
from transformers import default_data_collator

data_collator = default_data_collator

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [35]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [36]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([32, 384]), torch.Size([32, 384]))

In [37]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 85, 129,  85, 129,  85, 129,  85, 129,  85, 129,  85, 129,  85, 129,
          85, 129,  85, 129, 205,  36, 205,  36, 205,  36, 205,  36, 205,  36,
         205,  36, 263,  36], device='cuda:0'),
 tensor([ 90, 130,  90, 130,  90, 130,  90, 130,  90, 130,  90, 130,  90, 130,
          90, 130,  90, 130, 207,  36, 207,  36, 207,  36, 207,  36, 207,  36,
         207,  36, 286,  59], device='cuda:0'))

In [38]:
n_best_size = 20

In [39]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [40]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "paragraph"],
        examples["paragraph" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["idx"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [41]:
validation_features = datasets["validation"].map(prepare_validation_features, batched=True, remove_columns=datasets["validation"].column_names)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [42]:
raw_predictions = trainer.predict(validation_features)

In [43]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [44]:
max_answer_length = 30

In [45]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["paragraph"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 9.746748, 'text': 'use its influence over the Taliban'},
 {'score': 9.70196, 'text': 'to use its influence over the Taliban'},
 {'score': 8.680423, 'text': 'its influence over the Taliban'},
 {'score': 8.345432, 'text': 'influence over the Taliban'},
 {'score': 7.7139616, 'text': 'use its influence over the Taliban.'},
 {'score': 7.6691732, 'text': 'to use its influence over the Taliban.'},
 {'score': 6.6476364, 'text': 'its influence over the Taliban.'},
 {'score': 6.314113, 'text': 'Pakistan to use its influence over the Taliban'},
 {'score': 6.312646, 'text': 'influence over the Taliban.'},
 {'score': 5.8762546, 'text': 'Taliban'},
 {'score': 5.7182345, 'text': 'the Taliban'},
 {'score': 5.6134596,
  'text': 'persuade Pakistan to use its influence over the Taliban'},
 {'score': 4.9200945,
  'text': 'to persuade Pakistan to use its influence over the Taliban'},
 {'score': 4.3312936,
  'text': 'high-level effort to persuade Pakistan to use its influence over the Taliban'},


In [46]:
datasets["validation"][1]["answer"]

'Monetary rewards'

In [47]:
datasets["validation"][0]

{'paragraph': 'While this process moved along, diplomacy continued its rounds. Direct pressure on the Taliban had proved unsuccessful. As one NSC staff note put it, "Under the Taliban, Afghanistan is not so much a state sponsor of terrorism as it is a state sponsored by terrorists." In early 2000, the United States began a high-level effort to persuade Pakistan to use its influence over the Taliban. In January 2000, Assistant Secretary of State Karl Inderfurth and the State Department\'s counterterrorism coordinator, Michael Sheehan, met with General Musharraf in Islamabad, dangling before him the possibility of a presidential visit in March as a reward for Pakistani cooperation. Such a visit was coveted by Musharraf, partly as a sign of his government\'s legitimacy. He told the two envoys that he would meet with Mullah Omar and press him on  Bin Laden. They left, however, reporting to Washington that Pakistan was unlikely in fact to do anything," given what it sees as the benefits of 

In [48]:
import collections
from collections import defaultdict
examples = datasets["validation"]
features = validation_features
example_id_to_index ={}

for i in range(0,301):
  temp=features["example_id"][i]["answer"]
  example_id_to_index[i] = temp
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
  temp=features["example_id"][i]["answer"]
  features_per_example[example_id_to_index[temp]].append(i)

In [52]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
   
    example_id_to_index ={}

    for i in range(0,301):
      temp=features["example_id"][i]["answer"]
      example_id_to_index[i] = temp
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
      temp=features["example_id"][i]["answer"]
    features_per_example[example_id_to_index[temp]].append(i)

   
    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    
    for example_index, example in enumerate(tqdm(examples)):

        feature_indices = features_per_example[example_index]

        min_null_score = None 
        valid_answers = []
        
        context = example["paragraph"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        if not squad_v2:
            predictions[example["answer"]] = best_answer["text"]
        else:
          answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
          predictions[example["id"]] = answer

    return predictions

In [162]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 300 example predictions split into 484 features.


  0%|          | 0/300 [00:00<?, ?it/s]

In [178]:
mylist =[]
for x in range(0, 301):
    mylist.append(x)
print(mylist)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [179]:
for x in range(259, 301):
  final_predictions.update({x : "Null values"+str(x)})

In [180]:
len(final_predictions)

300

In [181]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [182]:
my_predictions = list(final_predictions.keys())
print(my_predictions)

['Children, Gerd, or Dorian Popa', 'Monetary rewards', 'Asking Pakistan to help the USA', 'Meeting with General Musharraf', 'President Clinton offered the moon in terms of better relations with the United States', 'A Presidential visit in March', 'Paper checks', 'Increasing trade with Pakistan', 'Persuading Pakistan to use its influence over the Taliban by dangling before him the possibility of a presidential visit in March as a reward for Pakistani cooperation', 'Romanian police or secret service', 'President Cliton', 'Karl Inderfurth', 'Michael Sheehan', 'The department of counterterrorism', 'Mullah Omar', 'No', 'Yes', 'Yes there was no disagreement between the two', "No, the CIA felt it would risk the president's life and they felt that Pakistan had not done enough to merit a presidential visit", '11.09.2020', 'March 25, 1999', 'March 15, 2000', '12.03.2019', 'January 2000', 'March 25, 2000', 'Direct pressure on Taliban had proved unsuccessful', 'The envoys reported that Pakistan wa

In [183]:
def f1_score(prediction, ground_truth):
    prediction_tokens = prediction.split()
    ground_truth_tokens = ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (prediction == ground_truth)

In [184]:
orig = list(datasets["validation"])
for i in range(0,len(orig)):
  orig[i] = orig[i]["answer"]

In [185]:
em = 0
for i in range(0,len(my_predictions)):
  if orig[i] == my_predictions[i]:
    em+=1 
print("Exact Match {}".format(em/len(orig)))

Exact Match 0.12333333333333334


In [186]:
from sklearn.metrics import f1_score
f1 = f1_score(orig, my_predictions, average='macro')
print("f1_score {}".format(f1))

f1_score 0.10522222222222222
